# Importando Bibliotecas e módulos necessários

In [1]:
import os
import sys
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), 'src')))
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from dss import dss
from powerflow.bus import bus_dictionary
from powerflow.batery import distbess_dictionary
from powerflow.generators import distgen_dictionary,add_gd
from powerflow.get_powers import get_dist_power
from powerflow.load import add_LoadToBus,add_load
from powerflow.text_comands import set_frequency,new_circuit,line_geometry,add_line,add_infos


# Diretórios dos dados


In [2]:
# Diretório de datasets
dir_datasets = 'C:\\Users\\joao9\\GitHub\\opendss\\datasets\\'

#Diretório de arquivos .dss
dir_dss = dir_datasets + 'arquivos_dss\\'

# Diretório de arquivos de geração PV
dir_pv = 'datasets\\generation_profiles\\'

# Diretório de arquivos de cargas
dir_cargas = 'datasets\\load_profiles\\'


# Definições iniciais do circuito

In [3]:

Text = dss.Text
Circuit = dss.ActiveCircuit
Solution = dss.ActiveCircuit.Solution

# Definições da MatPlotLib
plt.close('all')
plt.style.use('ggplot')
plt.rcParams["figure.figsize"] = (8,4)
plt.rcParams["figure.dpi"] = 100
plt.rcParams['font.family'] = 'DejaVu Sans'
plt.rcParams['font.sans-serif'] = ['cmr10']
plt.rcParams['axes.unicode_minus'] = False

#Potências máximas
PowerLoad=4
PowerPV=6 #Potência total dos módulos

# Definições do BESS
Pmin=0.320
BESS_P_MAX = 5
BESS_C_MAX = 12
SOC_MIN = 0.1 # 10%
SOC_MAX = 0.9 # 90%
BESS_E_MIN = BESS_C_MAX * SOC_MIN
BESS_E_MAX = BESS_C_MAX * SOC_MAX

# Definições do intervalo de simulação
# nconsumidores = len(os.listdir(dir_cargas))
nconsumidores = 3
nbarramentos = 4
horas = 24 #hours
intervalo = 30 #min

# Criação do modelo base do arquivo.dss

In [4]:
with open(dir_dss + 'ModelagemTeste.dss', 'w') as file:
    file.write(set_frequency(60) + '\n')
    file.write(new_circuit('ModelagemTeste', str(1).zfill(3), 0.22) + '\n')
    file.write(line_geometry(0.52,0.63,'km',3.338,3,3,[-120.00,0.00,60.00],[1000.00,1000.00,1000.00])+ '\n')
    for i in range(1,nbarramentos,1):
        file.write(add_line(i,str(i).zfill(3),str(i+1).zfill(3),25,'m',3)+ '\n')
    
    file.write(add_infos())

opendssmodel = dir_dss + 'ModelagemTeste.dss'

# Criação dos dicionários que irão armazenar as inforamções das cargas, geraçãoPV e bateria

In [5]:
profiles = os.listdir(dir_cargas)

buses = bus_dictionary(nbarramentos)
for i in range(3):
    bus = 'bus_'+str(i+1).zfill(3)
    name = profiles[i].replace('.csv','')
    add_LoadToBus(buses,bus,name,2,0.22,1,dir_cargas+profiles[i])

display(buses)

{'bus_001': {'MAC003982': {'phases': 2,
   'voltage': 0.22,
   'power': 1,
   'profile':                           datetime  Ppower    Qpower
   0      2012-07-06 12:00:00.0000000   0.194  0.000194
   1      2012-07-06 12:30:00.0000000   0.159  0.000159
   2      2012-07-06 13:00:00.0000000   0.161  0.000161
   3      2012-07-06 13:30:00.0000000   0.188  0.000188
   4      2012-07-06 14:00:00.0000000   0.144  0.000144
   ...                            ...     ...       ...
   28818  2014-02-27 22:30:00.0000000   0.123  0.000123
   28819  2014-02-27 23:00:00.0000000   0.133  0.000133
   28820  2014-02-27 23:30:00.0000000   0.146  0.000146
   28821  2014-02-28 00:00:00.0000000   0.193  0.000193
   28822  2014-02-28 00:00:00.0000000   0.193  0.000193
   
   [28823 rows x 3 columns]}},
 'bus_002': {'MAC003983': {'phases': 2,
   'voltage': 0.22,
   'power': 1,
   'profile':                           datetime  Ppower    Qpower
   0      2012-07-06 12:00:00.0000000   0.098  0.000098
   1     

# Garantindo que o Opendss esteja limpo

In [6]:
dss.ClearAll()
dss.Start(0)
Text.Command = "Clear"
Text.Command = f"Compile {opendssmodel}"

print(Circuit.AllBusNames)

['bus_001', 'bus_002', 'bus_003', 'bus_004']


# Fluxo de carga

In [9]:
column_power = ['datetime', 'bus','V', 'P', 'Q']
bus_power = pd.DataFrame(columns = column_power)



range_time = pd.date_range('2012-07-06 12:00:00', periods=horas*60/intervalo, freq=str(intervalo)+'T')
for time in range_time:
  new_command = ''

  #Acessa os barramentos do dicionário
  for bus in buses.keys():
    loads = buses[bus].keys()
    for load in loads:
      carga = buses[bus][load]
      new_command = new_command + '\n' + add_load(time,load,bus, carga['phases'], 
      carga['voltage'], carga['power'], carga['profile'])

  # for gen in distgen:
  #   new_command = new_command + '\n' + add_gd(time, gen, distgen[gen]['bus'], distgen[gen]['phases'], 
  #   distgen[gen]['conn'], distgen[gen]['voltage'], distgen[gen]['power'], distgen[gen]['profile'])



  dss.ClearAll()
  dss.Start(0)
  dss("Clear")
  dss(f"Compile {opendssmodel}")
  dss(new_command)
  dss("Set VoltageBases = [.22]")
  dss("CalcVoltageBases")
  dss("Set maxiterations = 500")
  dss('set mode = snap')
  dss("Solve")

  for bus in buses.keys():
    Circuit.SetActiveBus(bus)
    voltages = Circuit.ActiveBus.VMagAngle

    # Potências associadas ao barramento (P e Q)
    p_total, q_total = 0, 0
    conectado_bus = Circuit.ActiveBus.AllPDEatBus
    for elem in conectado_bus:
      Circuit.SetActiveElement(elem)
      powers = Circuit.PDElements.AllPowers  # Lista com potências [P1, Q1, P2, Q2, ...]
      p_total += sum(powers[::2])  # Somar potências ativas (P)
      q_total += sum(powers[1::2])  # Somar potências reativas (Q)

    bus_power = pd.concat([bus_power, pd.DataFrame([[time, bus, voltages[0], p_total, q_total]], columns=column_power)], ignore_index=True)

  

C:\Users\joao9\AppData\Local\Temp\ipykernel_21532\3270705722.py:6: FutureWarning: Non-integer 'periods' in pd.date_range, pd.timedelta_range, pd.period_range, and pd.interval_range are deprecated and will raise in a future version.
  range_time = pd.date_range('2012-07-06 12:00:00', periods=horas*60/intervalo, freq=str(intervalo)+'T')
C:\Users\joao9\AppData\Local\Temp\ipykernel_21532\3270705722.py:6: FutureWarning: 'T' is deprecated and will be removed in a future version, please use 'min' instead.
  range_time = pd.date_range('2012-07-06 12:00:00', periods=horas*60/intervalo, freq=str(intervalo)+'T')
C:\Users\joao9\AppData\Local\Temp\ipykernel_21532\3270705722.py:48: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  bus_power = pd.concat([bus_